# Session 3 - P4D Simulations

Let's import BattMo and some other packages into the kernel.

In [ ]:
using BattMo, GLMakie, Jutul

GLMakie.activate!(inline=false)

## Cylindrical cell

This example demonstrates how to set up, run and visualize a 3D cylindrical battery model. Let's first load the parameters and settings that we need and instantiate the model setup.


In [ ]:
cell_parameters     = load_cell_parameters(; from_default_set = "Chen2020")
cycling_protocol    = load_cycling_protocol(; from_default_set = "CCDischarge")
model_settings      = load_model_settings(; from_default_set = "P4D_cylindrical")
simulation_settings = load_simulation_settings(; from_default_set = "P4D_cylindrical")

model_setup = LithiumIonBattery(; model_settings)

Let's review and modify the cell parameters. We go through some of the geometrical and discretization parameters. We modify some of them to obtain a cell where the different components are easier to visualize

In [ ]:
cell_parameters["Cell"]["OuterRadius"] = 0.010 

We modify the current collector thicknesses, for visualization purpose

In [ ]:
cell_parameters["NegativeElectrode"]["CurrentCollector"]["Thickness"]    = 50e-6
cell_parameters["PositiveElectrode"]["CurrentCollector"]["Thickness"]    = 50e-6

The tabs are part of the current collectors that connect the electrodes to the external circuit. The location of the tabs is given as a fraction length, where the length is measured along the current collector in the horizontal direction, meaning that we follow the rolling spiral. Indeed, this is the relevant length to use if we want to dispatch the current collector in a equilibrated way, where each of them will a priori collect the same amount of current. In the following, we include three tabs with one in the middle and the other at a distance such that each tab will collect one third of the current.

In [ ]:
cell_parameters["NegativeElectrode"]["CurrentCollector"]["TabFractions"] = [0.5/3, 0.5, 0.5 + 0.5/3] 
cell_parameters["PositiveElectrode"]["CurrentCollector"]["TabFractions"] = [0.5/3, 0.5, 0.5 + 0.5/3]

We set the tab width to 2 mm.

In [ ]:
cell_parameters["NegativeElectrode"]["CurrentCollector"]["TabWidth"] = 0.002
cell_parameters["PositiveElectrode"]["CurrentCollector"]["TabWidth"] = 0.002

The angular discretization of the cell is determined by the number of angular grid points.

In [ ]:
simulation_settings["GridResolution"]["Angular"] = 30

Create the simulation object.

In [ ]:
sim = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings);

We preprocess the simulation object to retrieve the grids and coupling structure, which we want to visualize prior running the simulation.

In [ ]:
output = get_simulation_input(sim)
grids     = output[:grids]
couplings = output[:couplings]

#### Plotting the grid

Let's visualize the grids and couplings. Define a list of the component to iterate over in the ploting routin below.

In [ ]:
components = ["NegativeElectrode", "PositiveElectrode", "NegativeCurrentCollector", "PositiveCurrentCollector" ]
colors = [:gray, :green, :blue, :black]

We plot the components.

In [ ]:
for (i, component) in enumerate(components)
    if i == 1
        global fig, ax = plot_mesh(grids[component],
                            color = colors[i])
    else
        plot_mesh!(ax,
                   grids[component],
                   color = colors[i])
    end
end
fig

We plot the tabs, which couple the current collectors with the external circuits. The tabs will typically protude from the cell in the vertical directions but we can neglect this 3d feature in the simulation model. The tabs are then represented by horizontal faces at the top or bottom of the current collectors. In the figure below, they are plotted in red.

In [ ]:
components = [
    "NegativeCurrentCollector",
    "PositiveCurrentCollector"
]

for component in components
    plot_mesh!(ax, grids[component];
               boundaryfaces = couplings[component]["External"]["boundaryfaces"],
               color = :red)
end

fig #hide

ax.azimuth[] = 4.0
ax.elevation[] = 1.56

fig

#### Simulation

We reload the original parameters

In [ ]:
cell_parameters     = load_cell_parameters(; from_default_set = "Chen2020")
cycling_protocol    = load_cycling_protocol(; from_default_set = "CCDischarge")
model_settings      = load_model_settings(; from_default_set = "P4D_cylindrical")
simulation_settings = load_simulation_settings(; from_default_set = "P4D_cylindrical")

We adjust the parameters so that the simulation in this example is not too long (around a couple of minutes)

In [ ]:
cell_parameters["Cell"]["OuterRadius"] = 0.004 
cell_parameters["NegativeElectrode"]["CurrentCollector"]["TabFractions"] = [0.5] 
cell_parameters["PositiveElectrode"]["CurrentCollector"]["TabFractions"] = [0.5] 
cell_parameters["NegativeElectrode"]["CurrentCollector"]["TabWidth"]     = 0.002
cell_parameters["PositiveElectrode"]["CurrentCollector"]["TabWidth"]     = 0.002
simulation_settings["GridResolution"]["Angular"] = 8

We setup the simulation and run it

In [ ]:
sim = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings);
output = solve(sim; info_level = -1)

#### Visualize results

Let's plot the voltage and current profiles.

In [ ]:
plot_dashboard(output, "simple")

To show the results on the 3D geometry we have a really cool tool using Jutul that can be executed by the folowing function.

In [ ]:
plot_interactive_3d(output; colormap = :curl)

## Pouch cell

- C : electrolyte concentration
- Cp: particle concentration
- Cs: surface concentration
- Phi: potential

In [2]:
cell_parameters = load_cell_parameters(; from_default_set = "Xu2015")


{
    "Electrolyte" =>     {
        "TransferenceNumber" => 0.363
        "Description" => "1.5 mol/l LiPF6 dissolved in a mixture of ethylene carbonate (EC):  dimethyl carbonate (DMC) (1:1)"
        "DiffusionCoefficient" =>         {
            "FilePath" => "function_parameters_Xu2015.jl"
            "FunctionName" => "electrolyte_diffusivity_Xu_2015"
        }
        "IonicConductivity" =>         {
            "FilePath" => "function_parameters_Xu2015.jl"
            "FunctionName" => "electrolyte_conductivity_Xu_2015"
        }
        "Density" => 1210
        "ChargeNumber" => 1
        "Concentration" => 1500
    }
    "Cell" =>     {
        "ElectrodeGeometricSurfaceArea" => 0.007035
        "NominalCapacity" => 16.5
        "ElectrodeWidth" => 0.067
        "ElectrodeLength" => 0.105
        "Name" => "LP2770120"
        "Case" => "Pouch"
        "NominalVoltage" => 3.2
    }
    "Metadata" =>     {
        "Description" => "Parameter set of a commercial Type LP2770120 p

In [3]:

cycling_protocol = load_cycling_protocol(; from_default_set = "CCDischarge")
model_settings = load_model_settings(; from_default_set = "P4D_pouch")
simulation_settings = load_simulation_settings(; from_default_set = "P4D_pouch")

model_setup = LithiumIonBattery(; model_settings)

sim = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings);
output = solve(sim)

plot_interactive_3d(output; colormap = :curl)

✔️ Validation of ModelSettings passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 2 hours, 12 minutes as 163 report steps


Progress   1%|█                                          |  ETA: 0:47:40

Progress   7%|████                                       |  ETA: 0:07:46

Progress   9%|████                                       |  ETA: 0:06:36

Progress  10%|█████                                      |  ETA: 0:05:44

Progress  11%|█████                                      |  ETA: 0:05:03

Progress  12%|██████                                     |  ETA: 0:04:30

Progress  13%|██████                                     |  ETA: 0:04:03

Progress  15%|███████                                    |  ETA: 0:03:40

Progress  16%|███████                                    |  ETA: 0:03:21

Progress  17%|████████                                   |  ETA: 0:03:05

Progress  18%|████████                                   |  ETA: 0:02:51

Progress  20%|█████████                                  |  ETA: 0:02:38

Progress  21%|█████████                                  |  ETA: 0:02:27

Progress  21%|██████████              

╭────────────────┬───────────┬───────────────┬──────────╮
│ Iteration type │  Avg/step │  Avg/ministep │    Total │
│                │ 138 steps │ 138 ministeps │ (wasted) │
├────────────────┼───────────┼───────────────┼──────────┤
│ Newton         │   2.13043 │       2.13043 │  294 (0) │
│ Linearization  │   3.13043 │       3.13043 │  432 (0) │
│ Linear solver  │   2.13043 │       2.13043 │  294 (0) │
│ Precond apply  │       0.0 │           0.0 │    0 (0) │
╰────────────────┴───────────┴───────────────┴──────────╯
╭───────────────┬──────────┬────────────┬─────────╮
│ Timing type   │     Each │   Relative │   Total │
│               │       ms │ Percentage │       s │
├───────────────┼──────────┼────────────┼─────────┤
│ Properties    │   5.6942 │     3.80 % │  1.6741 │
│ Equations     │  30.0160 │    29.43 % │ 12.9669 │
│ Assembly      │  12.2251 │    11.99 % │  5.2813 │
│ Linear solve  │  27.4901 │    18.34 % │  8.0821 │
│ Linear setup  │   0.0000 │     0.00 % │  0.0000 │
│ Precond 